# 🎞️ Bonus: SAR Video & Subaperture Processing Magic ✨

Alright, by now we've learned how to generate **high-resolution SAR images** by combining **measurements** collected along the synthetic aperture. 📡🖼️  

We’ve also seen how we can divide the aperture into **smaller sub-apertures** to create multiple **“look” images**, which we then **average** to reduce speckle. 🎲➕🧩

---

## 🚪 Opening the Door to New Possibilities

But here’s the exciting part — this concept of **sub-aperture division** isn’t just useful for reducing speckle. It also unlocks some fascinating **exploitation techniques**. 🔓💡

Why? Because **each sub-aperture corresponds to measurements collected at a different time** as the radar platform moves during the collection. ⏱️

That means we actually capture the **temporal evolution** of the scene — how reflections **change over time**! 🔁

---

## 🎬 Enter: SAR Video

This leads us straight to the idea of a **SAR video**. 🎥

The method is **similar to multilooking** in terms of dividing the synthetic aperture into segments. But this time, we take a different approach:

- ❌ Instead of **averaging** the looks together...  
- ✅ We treat **each look as a frame** in a video!  

The result? A SAR-based video that **visualizes how the radar echoes evolve** over the course of the data collection. You’re essentially watching the scene unfold as the radar sees it — one sub-aperture at a time. 🛰️👁️‍🗨️🎞️

---

🎥 Let’s experiment with the **SAR video**, using a few **range-compressed datasets** from **Paris 🇫🇷** and **Rome 🇮🇹**!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.interpolate import interp1d
c = 299792458 # Speed of light (m/s)

def calculate_image_pixel_positions(scene_center_pos_ecef, center_of_aperture_pos, x_extent, y_extent, nx, ny):
    """
    Calculate image pixel coordinate positions in ECEF system.
    """
    # Construct local tangent-plane basis
    scene_center_pos = np.asarray(scene_center_pos_ecef, dtype=float)  # Scene center

    # z-axis: a vector perpendicular to the image (x,y) plane (surface normal)
    z_hat = scene_center_pos / np.linalg.norm(scene_center_pos)

    # define the line-of-sight vector from scene center to center of aperture
    slant_range_vector = scene_center_pos - center_of_aperture_pos 

    # Project it so that y-axis (ground-range) is purely horizontal along the ground plane
    ground_range_vector = slant_range_vector - np.dot(slant_range_vector, z_hat)*z_hat
    ground_range_unit_vector = np.linalg.norm(ground_range_vector)

    # y-axis is ground range
    y_hat = ground_range_vector / np.linalg.norm(ground_range_vector)

    # x-axis is cross-range (right-handed system), perpendicular to both y- and z-axis
    x_hat = np.cross(y_hat, z_hat)
    x_hat /= np.linalg.norm(x_hat)

    # Build local coordinate grid
    x_grid = np.linspace(x_extent[0], x_extent[1], nx)
    y_grid = np.linspace(y_extent[0], y_extent[1], ny)
    
    # Build an array of local coordinates, then convert to ECEF using scene center and image axis vectors
    XX, YY = np.meshgrid(x_grid, y_grid)  # shape (ny, nx) each
    # Flatten => shape (num_pixels,)
    X_flat = XX.ravel()
    Y_flat = YY.ravel()

    # Compute ECEF for each pixel
    scene_center = scene_center_pos.reshape((1,3)) 
    x_axis = x_hat.reshape((1,3)) # image cross-range axis
    y_axis = y_hat.reshape((1,3)) # image range axis

    # We use the x and y grids to compute the pixel positions in one go
    X_col = X_flat.reshape((-1,1))   # shape (num_pixels,1)
    Y_col = Y_flat.reshape((-1,1))   # shape (num_pixels,1)

    # This is an array of size (num_pixels,3) that has the (x,y,z) ECEF coordinates for each pixel location
    pixel_pos_ecef = scene_center + X_col*x_axis + Y_col*y_axis

    return pixel_pos_ecef, x_grid, y_grid

def sar_beamforming(
    slant_ranges,                   # 1D array of range bins in meters (shape = (num_range_samples,))
    range_compressed_data,          # Complex SAR data, shape = (num_pulses, num_range_samples)
    radar_pos_vs_time_ecef,         # Radar positions (num_pulses, 3) in ECEF
    center_of_aperture_pos,         # Radar position at the center of aperture (1, 3) in ECEF
    scene_center_pos_ecef,          # Scene center (3,) in ECEF
    wavelength,                     # Radar carrier wavelength (m)
    x_extent,                       # X (cross-range) extent in local coords around scene center (m)
    y_extent,                       # Y (ground range) extent in local coords around scene center (m)
    pixel_spacing_x,                             # Pixel spacing in x, meters (cross-range)
    pixel_spacing_y                              # Pixel spacing in in y, meters (ground range)
):
    """
    SAR Beamforming algorithm that:
      1) Builds a local tangent plane around 'scene_center_pos_ecef'.
         - z-axis is "up"
         - y-axis is ground range, along the line from scene center -> radar position at the mid-aperture point
         - x-axis is cross-range, along the filght path, completes a right-handed system
      2) Maps each local (x, y) pixel to ECEF postion (we need to have pixels and radar position in the same coordinate system)
      3) Uses backprojection beamforming to sum signals coherently to form each focused pixel value.

    Returns:
      sar_image:   2D complex array (ny, nx) with the focused image
      x_grid:  1D array of local x-coords
      y_grid:  1D array of local y-coords
    """

    x_size_m = np.abs(x_extent[-1] - x_extent[0]) # Image size in cross-range (m)
    y_size_m = np.abs(y_extent[-1] - y_extent[0]) # Image size in ground-range (m)
    nx = int(round( x_size_m / pixel_spacing_x )) # Number of pixels in cross-range
    ny = int(round( y_size_m / pixel_spacing_y )) # Number of pixels in ground-range
    
    # pixel_pos_ecef is an array that has the (x,y,z) ECEF coordinates for each pixel position
    pixel_pos_ecef, x_grid, y_grid = calculate_image_pixel_positions(
        scene_center_pos_ecef, 
        center_of_aperture_pos, 
        x_extent, 
        y_extent, 
        nx, 
        ny)

    # We'll accumulate the phasor sum for each pixel in a 1D array
    image_array_flat = np.zeros(pixel_pos_ecef.shape[0], dtype=np.complex128)

    # For each pulse, do vector calculations using the pixel array
    last_printed_percent = -1
    number_of_pulses = range_compressed_data.shape[0]
    for i_pulse in range(number_of_pulses):
        
        # Radar position at this pulse
        radar_xyz = radar_pos_vs_time_ecef[i_pulse, :]
        
        # Lets calculate an array that has the distance between each pixel and the radar at this pulse
        radar_to_pixel_vec = pixel_pos_ecef - radar_xyz # vectors from the radar position to each pixel
        # One way range to each pixel from this "virtual" element position
        range_to_all_pixels = np.sqrt(np.sum(radar_to_pixel_vec**2, axis=1))

        # This is the echo data of the current pulse
        pulse_echo_data = range_compressed_data[i_pulse, :] 
        
        # Pick the correct sample with cubic interpolation : This is more accurate than our previous nearest neighbor method
        real_interpolator = interp1d(slant_ranges, np.real(pulse_echo_data), kind='cubic', fill_value="extrapolate")
        imag_interpolator = interp1d(slant_ranges, np.imag(pulse_echo_data), kind='cubic', fill_value="extrapolate")
        # Interpolate I and Q separately and form the phasors
        echo_data_all_pixels = real_interpolator(range_to_all_pixels) + 1j*imag_interpolator(range_to_all_pixels)

        # Phase correction based on the range to each pixel
        phase_correction = 2.0 * np.pi * 2.0 * range_to_all_pixels / wavelength

        # Correct phases and accumulate pixel values
        image_array_flat += echo_data_all_pixels * np.exp(1j * phase_correction)
        
        # Print progress message every 5 percent, coz it's slow!        
        progress = int(round(100.0 * i_pulse / number_of_pulses))
        if progress % 25 == 0 and progress != last_printed_percent:
            print(f"SAR Beamforming Progress: {progress}%")
            last_printed_percent = progress

    # Reshape image to to (ny,nx) and normalise amplitude
    sar_image = image_array_flat.reshape((ny, nx)) / number_of_pulses

    return sar_image, x_grid, y_grid

### Here you can choose the dataset 🗼 🏛️

In [ ]:
# Load the radar data
#data_relative_path = Path("./data/range_compressed_SAR_dataset_1.npz") # Croissants for everyone!
data_relative_path = Path("./data/range_compressed_SAR_dataset_2.npz") # Bread and circuses for everyone!
data_path = data_relative_path.resolve()
sar_data_dict = np.load(data_path)

range_axis = sar_data_dict["range_vector"]
data_matrix = sar_data_dict["range_compressed_data"]
scene_center = sar_data_dict["scene_center_position"]
satellite_position_vs_pulse = sar_data_dict["satellite_position_vs_pulse"]
wavelength = sar_data_dict["wavelength"]
signal_bandwidth = sar_data_dict["chirp_bandwidth"]

number_of_pulses_in_data = data_matrix.shape[0]

🛠️ Feel free to **play around** with:
- The **number of frames** in the video 🎞️
- The **pixel spacing** of the image 📏🖼️

See how these settings affect the final result—and enjoy exploring the data! 🌌🔍

In [ ]:
# You can experiment with image sizes and amount of pixels. But be careful not to under- or oversample too much!
x_extent=(-300, 300) # Image extent in cross-range (m)
y_extent=(-300, 300) # Image extent in ground range (m)

# Radar position at the center of synthetic aperture (used to define ground range axis)
center_of_aperture_pos = satellite_position_vs_pulse[number_of_pulses_in_data//2,:]

pixel_spacing_x = 1.5 # np.abs(x_extent[-1] - x_extent[0]) / nx
pixel_spacing_y = 1.5 # np.abs(y_extent[-1] - y_extent[0]) / ny

# If you want to reconstruct the single look image for comparison, feel free to do so
#sar_image, x_grid, y_grid = sar_beamforming(range_axis, data_matrix, satellite_position_vs_pulse, center_of_aperture_pos, scene_center, 
#                                            wavelength, x_extent, y_extent,  pixel_spacing_x, pixel_spacing_y)

### Now you are in the director's seat! Let's make the SAR video! 🎥

In [ ]:
# CHOOSE THE NUMBER OF VIDEO FRAMES
number_of_frames = 10

# Let's determine how many pulses we have in each frame:
nb_pulses_per_frame = number_of_pulses_in_data // number_of_frames

for i_frame in range(number_of_frames):
    print("Calculating video frame number", i_frame+1, "of", number_of_frames)
    # Calculate which pulses from the data to use for this look
    pulse_start = i_frame*nb_pulses_per_frame
    pulse_end = pulse_start + nb_pulses_per_frame
    print("Start, end pulse index for this frame:", pulse_start, pulse_end)
    # Pick the data (i.e. the virtual elements) that correspond to this look
    frame_data = data_matrix[pulse_start:pulse_end,:]
    frame_radar_pos = satellite_position_vs_pulse[pulse_start:pulse_end:,:]
    frame_image, _, _ = sar_beamforming(range_axis, frame_data, 
                                       frame_radar_pos, center_of_aperture_pos,
                                       scene_center, wavelength, x_extent, y_extent, 
                                       pixel_spacing_x, pixel_spacing_y)
    if i_frame == 0:
        # Make an array for the frames
        video_frames = np.zeros((frame_image.shape[0], frame_image.shape[1], number_of_frames))
    # Collect the frame in the video array
    video_frames[:,:,i_frame] = np.abs(frame_image)
    

### Let's play the video! 🎞️

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Adjust display. Let's use log scale to see details more clearly
dynamic_range_dB = 25 # FEEL FREE TO ADJUST THIS VALUE TO DISPLAY THE IMAGE
max_val = np.max(10*np.log10(np.abs(video_frames.ravel())))
min_val = max_val - dynamic_range_dB 

# Create the figure and initial frame
fig, ax = plt.subplots()
im = ax.imshow(10*np.log10(np.abs(video_frames[:, :, 0])), aspect='auto',
               cmap='jet', vmin=min_val, vmax=max_val, animated=True)
ax.axis('off')
ax.set_title("SAR Video Playback")

# Update function
def update(frame):
    im.set_array(10*np.log10(np.abs(video_frames[:, :, frame])))
    return [im]

# Create animation
ani = FuncAnimation(fig, update, frames=number_of_frames, interval=250, blit=False)

# Display animation in notebook
HTML(ani.to_jshtml())

## 🌈 **Colorised Subaperture Imaging: A New Dimension of Insight**

Dividing our synthetic aperture into **subapertures** doesn’t just open the door to SAR video — it also enables a fascinating way to explore how targets **reflect energy from different angles**. 🛰️🔁  

Remember: each subaperture captures the scene from a **slightly different perspective** and at a **different moment in time**. This means we can examine the **angular dependence of reflectivity** — a key characteristic of many real-world targets. 📐💡

---

### 🪞 **Angular Reflectivity: Not All Targets Reflect Equally**

Most objects on Earth do **not** reflect radar energy equally in all directions (i.e., they’re not isotropic reflectors). Instead, their brightness in SAR images depends on the **viewing angle**. Here’s a classic example:

- Imagine a **flat metal plate** — like the **side of a car** or a **rooftop**.  
- When viewed **obliquely**, radar waves reflect away (per the **law of reflection**) and **very little energy returns to the radar**.  
- That’s why **calm water** often appears **black in SAR images** — it reflects energy away from the sensor, not back toward it. 🌊🚫📡

But when the radar **views the surface head-on**, i.e., along its **surface normal**, it produces a **strong specular reflection** — resulting in a **bright spot** in the image.  
These types of **direction-dependent reflections** are typical of **man-made structures** with regular shapes. 🏠🚗🏗️

---

### 🎨 **Encoding Angular Scattering in Color**

One creative way to visualize this angular dependence is through **colorised subaperture imaging**:

- 🔪 We divide the full synthetic aperture into **several subapertures**.  
- 🟥🟩🟦 For example, we can split the aperture into **three segments**: beginning, middle, and end.  
- For each segment, we create a **subaperture image** (with lower cross-range resolution than the full aperture).  
- Then, we assign the **magnitude of each subaperture image** to a **color channel**:
  - **Red** = early subaperture  
  - **Green** = middle  
  - **Blue** = late  

The result? A **false-color RGB SAR image** that shows **where in the aperture the reflection occurred strongest**. 🌈🖼️

---

### 🔍 **How to Read a Colorised Subaperture Image**

- **Gray or white pixels** = the target reflected **evenly across all subapertures**.  
- **Red regions** = the target reflected **mostly in the first part** of the aperture (early viewing angle).  
- **Blue regions** = the strongest return came **late in the aperture** (from a different angle).  
- **Color transitions** in the scene tell us that **target reflectivity is changing with angle** — very useful for characterizing structures, vehicles, and complex materials. 🏢 🚙

In practice, we often use **more than three subapertures** and apply a **continuous color mapping scheme**, but the core idea remains the same:  
👉 **Color = dominant subaperture contribution** at each pixel.

---

This is a powerful way to visualize **directional scattering behavior** — and a beautiful example of how SAR can give us **much more than just black-and-white images**. 🌈

### Let's make the color image by dividing the aperture in three color frames! 🌈

In [ ]:
# Let's divide the aperture into three color frames
number_of_frames = 3
# Let's determine how many pulses we have in each frame:
nb_pulses_per_frame = number_of_pulses_in_data // number_of_frames

pixel_spacing_x = 1.5 # m
pixel_spacing_y = 1.5 # m

for i_frame in range(number_of_frames):
    print("Calculating CSI frame number", i_frame+1)
    # Calculate which pulses from the data to use for this look
    pulse_start = i_frame*nb_pulses_per_frame
    pulse_end = pulse_start + nb_pulses_per_frame
    print("Start, end pulse index for this frame:", pulse_start, pulse_end)
    # Pick the data (i.e. the virtual elements) that correspond to this look
    frame_data = data_matrix[pulse_start:pulse_end,:]
    frame_radar_pos = satellite_position_vs_pulse[pulse_start:pulse_end:,:]
    frame_image, _, _ = sar_beamforming(range_axis, frame_data, 
                                       frame_radar_pos, center_of_aperture_pos,
                                       scene_center, wavelength, x_extent, y_extent, 
                                       pixel_spacing_x, pixel_spacing_y)
    if i_frame == 0:
        csi_frames = np.zeros((frame_image.shape[0], frame_image.shape[1], number_of_frames))
    # Accumulate the squared magnitude (power) of the image
    csi_frames[:,:,i_frame] = np.abs(frame_image)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def normalize_band(band):
    """Normalize band to [0, 1] range."""
    return (band - band.min()) / (band.max() - band.min() + 1e-8)

def clip_and_normalize(band, pmin=0, pmax=95):
    """Clip extreme values and normalize to [0, 1]."""
    vmin, vmax = np.percentile(band, [pmin, pmax])
    band_clipped = np.clip(band, vmin, vmax)
    return normalize_band(band_clipped)

def to_log_normalized(image, pmin=0, pmax=95):
    """Apply log compression, clip extremes, and normalize."""
    log_img = 10 * np.log10(image + 1e-6)
    return clip_and_normalize(log_img, pmin, pmax)

def to_linear_normalized(image, pmin=0, pmax=95):
    """Clip and normalize power image without log compression."""
    return clip_and_normalize(image, pmin, pmax)

# Compute power from complex SAR data
power_r = np.abs(csi_frames[:, :, 0])**2
power_g = np.abs(csi_frames[:, :, 1])**2
power_b = np.abs(csi_frames[:, :, 2])**2

# Feel free to play around with the clipping settings in the image display. 
# (They are percentages according to which we clip the histogram.)

# Create log-normalized RGB image (with clipped amplitude for nicer visualization)
log_clip_low = 15
log_clip_high = 95
r_log = to_log_normalized(power_r, log_clip_low, log_clip_high)
g_log = to_log_normalized(power_g, log_clip_low, log_clip_high)
b_log = to_log_normalized(power_b, log_clip_low, log_clip_high)
rgb_log = np.stack((r_log, g_log, b_log), axis=-1)

# Create linear-normalized RGB image (with clipped amplitude for nicer visualization)
linear_clip_low = 0
linear_clip_high = 96
r_linear = to_linear_normalized(power_r, linear_clip_low, linear_clip_high)
g_linear = to_linear_normalized(power_g, linear_clip_low, linear_clip_high)
b_linear = to_linear_normalized(power_b, linear_clip_low, linear_clip_high)
rgb_linear = np.stack((r_linear, g_linear, b_linear), axis=-1)

# Plot: Log Scale
plt.imshow(rgb_log, aspect='auto')
plt.title("Colorized Subaperture Image (Log Scale)")
plt.axis('off')
plt.show()

# Plot: Linear Scale
plt.imshow(rgb_linear, aspect='auto')
plt.title("Colorized Subaperture Image (Linear Scale)")
plt.axis('off')
plt.show()